## Pegando os dados para trabalhar com eles
Aqui estarei pegando alguns dados do quandl, como o stock price da google. Através disso, faremos uma análise

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import quandl

api_key = 'yQaxzc5rcgC8Hyt8SSdz'

df = quandl.get("WIKI/GOOGL", authtoken=api_key)
df = df[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Volume']]

df['HL_PCT'] = (df['Adj. High'] - df['Adj. Low']) / df['Adj. Low'] * 100.0
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0 

df.head()

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Volume,HL_PCT,PCT_change
Date,,,,,,,
2004-08-19,50.159839,52.191109,48.128568,50.322842,44659000.0,8.441017,0.324968
2004-08-20,50.661387,54.708881,50.405597,54.322689,22834300.0,8.537313,7.227007
2004-08-23,55.551482,56.915693,54.693835,54.869377,18256100.0,4.062357,-1.227880
2004-08-24,55.792225,55.972783,51.945350,52.597363,15247300.0,7.753210,-5.726357
2004-08-25,52.542193,54.167209,52.100830,53.164113,9188600.0,3.966115,1.183658


Agora precisamos normalizar os dados, caso haja algum NaN

In [2]:
df.fillna(-1, inplace=True)

In [3]:
import math

forecast_col = 'Adj. Close'

forecast_out = int(math.ceil(0.01*len(df)))

forecast_out

33

Neste exemplo, vamos tentar prever 10% do dataframe, oou seja usaremos 10 dias para fazer a previsão. Forecast_out é a quantidade de dados que vamos 'arrastar' os dados

## Label
Label é a saída, que no caso é a previsão. Faremos uma coluna label no df. Ela vai ser igual ao shift da coluna de previsão. O shift serve para arrastar os dados, para cima ou para baixo. É necessário então fazer esse shift com o forecast_out, que vai ser um inteiro

O forecast_out nesse caso é 26. Nós reajustamos o dataset para 26 dias depois, o que simula uma previsão, já que a coluna label é igual à adj.Close de 26 dias depois.

In [6]:
df['label'] = df[forecast_col].shift(-forecast_out)
df.dropna(inplace=True)
df.head()

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Volume,HL_PCT,PCT_change,label
Date,,,,,,,,
2004-08-19,50.159839,52.191109,48.128568,50.322842,44659000.0,8.441017,0.324968,68.752232
2004-08-20,50.661387,54.708881,50.405597,54.322689,22834300.0,8.537313,7.227007,69.639972
2004-08-23,55.551482,56.915693,54.693835,54.869377,18256100.0,4.062357,-1.227880,69.078238
2004-08-24,55.792225,55.972783,51.945350,52.597363,15247300.0,7.753210,-5.726357,67.839414
2004-08-25,52.542193,54.167209,52.100830,53.164113,9188600.0,3.966115,1.183658,68.912727


Em ML, 'X' = Features e 'y' são os labels

In [5]:
from sklearn import preprocessing, svm, cross_validation
from sklearn.linear_model import LinearRegression

